In [ ]:
# %%
# import tensorflow as tf

# from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd 
import skimage.measure

import random
import copy
import math
import cv2
import os

from IPython.display import clear_output

# %%
idx = 7
if True:
# for idx in range(13):
  
  div = 32
  step = int(32/4)
  #      0    1   2   3   4  5   6  7  8  9 10 11 12 13 14 15
  red = [11, 12, 14, 14, 14, 16, 6, 7, 7, 7, 7, 7, 7, 5, 5, 6] # Check the estimated_scales.yaml -> the resolution has to be 0.4 meters/pixel
  red = red[idx]
  #                   0           1             2         3        4        5        6       7         8        9        10        11       12       13        14       15
  location_list = ['bookstore', 'bookstore', 'coupa', 'coupa', 'coupa', 'coupa', 'gates', 'gates', 'hyang', 'hyang', 'hyang', 'hyang', 'little', 'nexus', 'nexus', 'deathCircle']
  number_list =   ['0',               '4',      '0',      '1',      '2',     '3',    '2',    '6',     '2',     '3',     '4',     '10',    '3',     '0',     '1',        '0']

  # semantic class cash is used also for bike parking spots
  # sem_dict = ['cash', 'entrance', 'light', 'sit', 'stairs', 'trash', 'tree','restricted','grass','intersection','shadow']
  sem_dict = ['bicycle_road', 'building', 'entrance', 'grass', 'obstacle', 'parking', 'pedestrian_road', 'tree', 'vehicle_road']
  
  chans = len(sem_dict)

  lut_in = [0, 20, 50, 100, 150, 255]
  lut_out = [0, 100, 180, 220, 240, 255]
  lut_8u = np.interp(np.arange(0, 256), lut_in, lut_out).astype(np.uint8)

  location = location_list[idx]
  number = number_list[idx]
  map_name = 'stanford_'+location+number
  directory_dataset = '/'+location+'/video'+number+'/'
  print(map_name)

  # %%
  # Code to check if the red variable is correct to scale to the correct homography. Check on google maps and note that each pixel should be 0.4 meters (32 pixels = 12.8m)

  cap = cv2.VideoCapture('/data/placido/Stanford_Drone_Dataset/video'+directory_dataset+'video.mp4')
  df = pd.read_csv('/data/placido/Stanford_Drone_Dataset/annotations'+directory_dataset+'annotations.txt', delimiter=' ', header=None)
  
  # Check if camera opened successfully
  if (cap.isOpened()== False): 
    print("Error opening video stream or file")
  cap.set(cv2.CAP_PROP_POS_FRAMES, 1000)
  ret, frame = cap.read()
  # Create the output directory if it doesn't exist
  os.makedirs('maps/semantics/'+map_name, exist_ok=True)
  cv2.imwrite('maps/semantics/'+map_name+'/reference.png', frame)
  ref_frame = frame
  cap.set(cv2.CAP_PROP_POS_FRAMES, 0)

  print(frame.shape)
  h = frame.shape[0]
  w = frame.shape[1]

  frame = skimage.measure.block_reduce(frame, (red,red,1), np.max)

  print(frame.shape)

  plt.figure()
  plt.imshow(frame)
  plt.show()

  dt = 1/cap.get(cv2.CAP_PROP_FPS)
  dt = 0.5
  # When everything done, release the video capture object
  cap.release()

  data = np.zeros((h,w))
  data_vel = np.zeros((h,w))
  data_stops = np.zeros((h,w))
  pos = {'x': 0.0, 'y': 0.0}
  vel = {'x': 0.0, 'y': 0.0}

  ratioy = (df[4].max())/h
  ratiox = (df[3].max())/w

  ped = df[df[9]=='Pedestrian']# filter only people
  ped = ped.reset_index()

  alpha = 0.2
  for trackid in ped[0].unique():
    ce = ped[ped[0]==trackid]
    ce = ce.reset_index()

    data_temp = np.zeros((h,w))
    vel_temp = np.zeros((h,w))
    stop_temp = np.zeros((h,w))
    for index, el in ce.iterrows():
      centerOfCircle = (int((el[2]+el[4])/2/ratioy),int((el[1]+el[3])/2/ratiox))
      if index==0:
        pos = {'x': centerOfCircle[0],'y': centerOfCircle[1]}
      vel = {'x': alpha*(centerOfCircle[0]-pos['x'])/dt/red + (1-alpha)*vel['x'],'y': alpha*(centerOfCircle[1]-pos['y'])/dt/red + (1-alpha)*vel['y']}
      pos = {'x': centerOfCircle[0],'y': centerOfCircle[1]}

      if centerOfCircle[0] < h and centerOfCircle[1] < w and (el[6] == 0 or el[7] == 1):
        data_temp[centerOfCircle] = 1
        if index > 0:
          vel_temp[centerOfCircle] = math.sqrt(vel['x']**2+vel['y']**2)
          if(vel_temp[centerOfCircle] < 1e-30):
            stop_temp[centerOfCircle] = 1

    if np.max(vel_temp) < 1.5:
      data_vel = data_vel * (vel_temp == 0) + data_vel * (vel_temp > 0)*0.5 + vel_temp * (data_vel > 0)*0.5 + vel_temp * (data_vel == 0)
      data = data + data_temp
      data_stops = data_stops + stop_temp
    # else:
    #   print('VEL EXEEDS 2')

  # data = np.clip(data,0,np.max(data)*0.2)/(np.max(data)*0.2)
  # data = cv2.LUT((data/np.max(data)*255).astype(np.uint8), lut_8u).astype(float)/255
  ref_frame_red = skimage.measure.block_reduce(ref_frame, (red,red,1), np.max)/510+0.5
  plt.figure(figsize=(6,5))
  data_show = data/np.max(data)
  plt.imshow(np.multiply(ref_frame/510+0.5, np.stack((np.full(data_show.shape,1),1-data_show,1-data_show),axis=2)), vmin=0, vmax=255)
  plt.show()
  data = skimage.measure.block_reduce(data, (red,red), np.max)
  print(f'data max: {np.max(data)}')
  print(f'data min: {np.min(data)}')
  data_show = data/np.max(data)

  # data = np.subtract(data, np.full((data.shape[0], data.shape[1]), np.min(data)))/(np.max(data)-np.min(data))

  plt.figure(figsize=(6,5))
  # plt.imshow(np.multiply(np.stack((map_var[:,:,0],map_var[:,:,0],map_var[:,:,0]),axis=2), np.stack((np.full(data.shape,1),1-data,1-data),axis=2)))
  plt.imshow(np.multiply(ref_frame_red, np.stack((np.full(data_show.shape,1),1-data_show,1-data_show),axis=2)))
  plt.show()

  np.savetxt('maps/semantics/'+map_name+'/humandensity-'+map_name+'-new.csv', data*255, delimiter=',', fmt='%3d')

  # -----------------------------------------------------------------------------------

  data_vel = skimage.measure.block_reduce(data_vel, (red,red), np.max)
  print(f'data_vel max: {np.max(data_vel)}')
  print(f'data_vel min: {np.min(data_vel)}')
  data_vel_show = data_vel/np.max(data_vel)
  # data_vel = np.subtract(data_vel, np.full((data.shape[0], data.shape[1]), np.min(data_vel)))/(np.max(data_vel)-np.min(data_vel))

  plt.figure(figsize=(6,5))
  # plt.imshow(np.multiply(np.stack((map_var[:,:,0],map_var[:,:,0],map_var[:,:,0]),axis=2), np.stack((np.full(data_vel.shape,1),1-data_vel,1-data_vel),axis=2)))
  plt.imshow(np.multiply(ref_frame_red, np.stack((np.full(data_vel_show.shape,1),1-data_vel_show,1-data_vel_show),axis=2)))
  plt.show()

  np.savetxt('maps/semantics/'+map_name+'/humandensity-'+map_name+'-vel.csv', data_vel*255, delimiter=',', fmt='%3d')

  # -----------------------------------------------------------------------------------

  data_stops = skimage.measure.block_reduce(data_stops, (red,red), np.max)
  print(f'data_stops max: {np.max(data_stops)}')
  print(f'data_stops min: {np.min(data_stops)}')
  data_stops_show = data_stops/np.max(data_stops)
  # data_stops = np.subtract(data_stops, np.full((data.shape[0], data.shape[1]), np.min(data_stops)))/(np.max(data_stops)-np.min(data_stops))

  plt.figure(figsize=(6,5))
  # plt.imshow(np.multiply(np.stack((map_var[:,:,0],map_var[:,:,0],map_var[:,:,0]),axis=2), np.stack((np.full(data_stops.shape,1),1-data_stops,1-data_stops),axis=2)))
  plt.imshow(np.multiply(ref_frame_red, np.stack((np.full(data_stops_show.shape,1),1-data_stops_show,1-data_stops_show),axis=2)))
  plt.show()

  np.savetxt('maps/semantics/'+map_name+'/humandensity-'+map_name+'-stop.csv', data_stops*255, delimiter=',', fmt='%3d')